In [51]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

In [52]:
# printing data pulled in
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


Player Count

In [53]:
# pulling only unique SN's and counting number of players
Player_Counts = purchase_data['SN'].nunique()
Player_dicts = [{"Total Players": Player_Counts}]

# Placing in Dataframe
Player_Counts_DF = pd.DataFrame(Player_dicts)

Player_Counts_DF.head()

,Total Players
0,576


Purchasing Analysis (Total)

In [54]:
# Calculating on aggregate Dataframe the average price, number of items, number of purchases and total revenue 

Items = purchase_data["Item ID"].nunique()
PurchaseID = purchase_data["Purchase ID"].nunique()
average_price = purchase_data["Price"].mean()
total_revenue = purchase_data["Price"].sum()

# Pulling into a Datafram
Summary_dicts = [{"Items": Items,"Purchases": PurchaseID,"Average Price": average_price,"Revenue": total_revenue,}]
Summary_DF = pd.DataFrame(Summary_dicts)
Summary_DF.head()


,Average Price,Items,Purchases,Revenue
0,3.050987,183,780,2379.77


Gender Demographics

In [55]:
# Creating new DataFrame which groups by Gender and pulls in only the unique SN's
gender = pd.DataFrame(purchase_data.groupby("Gender")["SN"].nunique())

# Calculating percentages and counts by gender 
gender["Percentage of Players"] = gender / Player_Counts * 100
gender = gender.rename(columns={"SN" : "Total Counts"})
gender




,Total Counts,Percentage of Players
Gender,,
Female,81,14.062500
Male,484,84.027778
Other / Non-Disclosed,11,1.909722


Purchasing Analysis (Gender)

In [56]:
# Pulling only SN, Gender and Price columns and grouping by Gender 
purchase_analysis = purchase_data[["SN", "Gender", "Price"]]
purchase_analysis_groupby = purchase_analysis.groupby(["Gender"])

# Calcs based on none unique or total data 
purchase_analysis_count = purchase_analysis_groupby["SN"].count()
purchase_analysis_sum = purchase_analysis_groupby["Price"].sum()
purchase_analysis_avgprice = purchase_analysis_groupby["Price"].mean()

# creating new dataframe to only calc based on unique SN for each gender type
purchase_analysis_group_person = pd.DataFrame(purchase_analysis.groupby(["Gender","SN"]).sum())
purchase_analysis_group_person = purchase_analysis_group_person.groupby(["Gender"])["Price"].mean()

In [57]:
#Creating new dataframe

purchase_analysis_df = pd.DataFrame({"Purchase Count" : purchase_analysis_count,
                                    "Average Purchase Price" : purchase_analysis_avgprice,
                                    "Total Purchase Value" : purchase_analysis_sum,
                                    "Avg Total Purchase Per Person" : purchase_analysis_group_person})
# Formating on new Dataframe
purchase_analysis_df["Average Purchase Price"] = purchase_analysis_df["Average Purchase Price"].map("${:.2F}".format)
purchase_analysis_df["Total Purchase Value"] = purchase_analysis_df["Total Purchase Value"].map("${:.2F}".format)
purchase_analysis_df["Avg Total Purchase Per Person"] = purchase_analysis_df["Avg Total Purchase Per Person"].map("${:.2F}".format)

purchase_analysis_df.head()


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase Per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


Age Demographics

In [58]:
# Calculating and checking unique SN count to use for binning exercise
new_purchase_data = purchase_data.drop_duplicates("SN")
new_purchase_data.count()

Purchase ID    576
SN             576
Age            576
Gender         576
Item ID        576
Item Name      576
Price          576
dtype: int64

In [59]:
# Bin variables based on age
bins = [-1, 9, 14, 19, 24, 29, 34, 39, 101]
bin_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# Creating new columns based on binning criteria based on unique SN's
new_purchase_data["Age Range Summary"] = pd.cut(new_purchase_data["Age"],bins, labels=bin_names)
purchase_data_age = new_purchase_data.groupby("Age Range Summary")

# Calcs 
Age_Count = purchase_data_age["Age"].count()
Age_Percentage = Age_Count / Player_Counts * 100

#Creating a new Dataframe
Age_Count_DF = pd.DataFrame({"Total Count" : Age_Count, "Percentage of Players" : Age_Percentage})
Age_Count_DF


C:\Users\Dgrimm\Anaconda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Total Count,Percentage of Players
Age Range Summary,,
<10,17,2.951389
10-14,22,3.819444
15-19,107,18.576389
20-24,258,44.791667
25-29,77,13.368056
30-34,52,9.027778
35-39,31,5.381944
40+,12,2.083333


In [60]:
# Creating new columns based on binning criteria based on all data
purchase_data["Age Range Summary"] = pd.cut(purchase_data["Age"],bins, labels=bin_names)
purchase_data_age = purchase_data.groupby("Age Range Summary")

# Creating new Dataframe based on all data and checking its pulling in right
purchase_analysis_group_person = pd.DataFrame(purchase_data.groupby(["Age Range Summary","SN"]).sum())
purchase_analysis_group_person = purchase_analysis_group_person.groupby(["Age Range Summary"])["Price"].mean()
purchase_analysis_group_person


Age Range Summary
<10      4.537059
10-14    3.762727
15-19    3.858785
20-24    4.318062
25-29    3.805195
30-34    4.115385
35-39    4.763548
40+      3.186667
Name: Price, dtype: float64

In [61]:
# Calcs

PurchaseID_Count = purchase_data_age["Purchase ID"].count()
PurchaseID_Sum = purchase_data_age["Price"].sum()
PurchaseID_Avg = purchase_data_age["Price"].mean()
# Building DataFrames
age_purchase_analysis = pd.DataFrame({"Purchase Count" : PurchaseID_Count, 
                                    "Average Purchase Prices" : PurchaseID_Avg, 
                                    "Total Purchase Value" : PurchaseID_Sum, 
                                    "Avg Total Purchase per Person" : purchase_analysis_group_person})
# Formatting
age_purchase_analysis["Average Purchase Prices"] = age_purchase_analysis["Average Purchase Prices"].map("${:.2F}".format)
age_purchase_analysis["Total Purchase Value"] = age_purchase_analysis["Total Purchase Value"].map("${:.2F}".format)
age_purchase_analysis["Avg Total Purchase per Person"] = age_purchase_analysis["Avg Total Purchase per Person"].map("${:.2F}".format)

age_purchase_analysis

,Purchase Count,Average Purchase Prices,Total Purchase Value,Avg Total Purchase per Person
Age Range Summary,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [62]:
# Calcs
Spenders_Analysis = purchase_data.groupby("SN")
Spenders_Count = Spenders_Analysis["Purchase ID"].count()
Spenders_Mean = Spenders_Analysis["Price"].mean()
Spenders_Total = Spenders_Analysis["Price"].sum()

# Building DataFrame
Spenders_DF = pd.DataFrame({ "Purchase Count" : Spenders_Count, 
                            "Average Purchase Price" : Spenders_Mean,
                           "Total Purchase Value" : Spenders_Total})
# Sorting and formatting
Spenders_DF = Spenders_DF.sort_values(["Total Purchase Value"], ascending = False)
Spenders_DF["Average Purchase Price"] = Spenders_DF["Average Purchase Price"].map("${:.2F}".format)
Spenders_DF["Total Purchase Value"] = Spenders_DF["Total Purchase Value"].map("${:.2F}".format)

Spenders_DF.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [63]:
# Pulling in only Item ID, Item Name and Price
MostPopular_Items = purchase_data[["Item ID","Item Name","Price"]]
# Grouping by Item ID and Item Name
MostPopular_Items_Group = MostPopular_Items.groupby(["Item ID", "Item Name"])
# Calc
MostPopular_Items_Count = MostPopular_Items_Group["Item ID"].count()
MostPopular_Items_Avg = MostPopular_Items_Group["Price"].mean()
MostPopular_Items_Total = MostPopular_Items_Group["Price"].sum()
# DataFrame
MostPopular_Items_DF = pd.DataFrame({"Purchase Count" : MostPopular_Items_Count,
                                    "Item Price" : MostPopular_Items_Avg,
                                    "Total Purchase Value" : MostPopular_Items_Total})
# Sorting and formatting
MostPopular_Items_DF = MostPopular_Items_DF.sort_values(["Purchase Count"], ascending = False)

MostPopular_Items_DF["Item Price"] = MostPopular_Items_DF["Item Price"].map("${:.2F}".format)
MostPopular_Items_DF["Total Purchase Value"] = MostPopular_Items_DF["Total Purchase Value"].map("${:.2F}".format)


MostPopular_Items_DF

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16
103,Singed Scalpel,8,$4.35,$34.80
75,Brutality Ivory Warmace,8,$2.42,$19.36
72,Winter's Bite,8,$3.77,$30.16
60,Wolf,8,$3.54,$28.32


In [64]:
# Sorting based on Most profitable Item
# Grouping by Item ID and Item Name
MostPopular_Items_Groups = MostPopular_Items.groupby(["Item ID", "Item Name"])
# Calcs
MostPopular_Items_Count = MostPopular_Items_Groups["Item ID"].count()
MostPopular_Items_Avg = MostPopular_Items_Groups["Price"].mean()
MostPopular_Items_Total = MostPopular_Items_Groups["Price"].sum()
#Building the DataFrame
MostPopular_Items_DF2 = pd.DataFrame({"Purchase Count" : MostPopular_Items_Count,
                                    "Item Price" : MostPopular_Items_Avg,
                                    "Total Purchase Value" : MostPopular_Items_Total})
#Sort and formating
profit_analaysis2 = MostPopular_Items_DF2.sort_values(["Total Purchase Value"], ascending = False)
MostPopular_Items_DF2["Item Price"] = MostPopular_Items_DF2["Item Price"].map("${:.2F}".format)
MostPopular_Items_DF2["Total Purchase Value"] = MostPopular_Items_DF2["Total Purchase Value"].map("${:.2F}".format)

profit_analaysis2

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
82,Nirvana,9,4.90,44.10
145,Fiery Glass Crusader,9,4.58,41.22
92,Final Critic,8,4.88,39.04
103,Singed Scalpel,8,4.35,34.80
59,"Lightning, Etcher of the King",8,4.23,33.84
108,"Extraction, Quickblade Of Trembling Hands",9,3.53,31.77
78,"Glimmer, Ender of the Moon",7,4.40,30.80
72,Winter's Bite,8,3.77,30.16
